<a href="https://colab.research.google.com/github/joanby/tensorflow2/blob/master/Collab%2013%20-%20Preparar%20un%20modelo%20de%20TensorFlow%20para%20móviles%20con%20TensorFlow%20Lite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Paso 1: Instalar las dependencias y configurar el entorno de trabajo

In [1]:
!pip install tensorflow-gpu==1.13.1

     |████████████████████████████████| 345.2MB 51kB/s 
     |████████████████████████████████| 3.2MB 28.7MB/s 
     |████████████████████████████████| 368kB 34.0MB/s 
ERROR: tensorflow 1.14.0 has requirement tensorboard<1.15.0,>=1.14.0, but you'll have tensorboard 1.13.1 which is incompatible.
ERROR: tensorflow 1.14.0 has requirement tensorflow-estimator<1.15.0rc0,>=1.14.0rc0, but you'll have tensorflow-estimator 1.13.0 which is incompatible.
  Found existing installation: tensorboard 1.14.0
    Uninstalling tensorboard-1.14.0:
      Successfully uninstalled tensorboard-1.14.0
  Found existing installation: tensorflow-estimator 1.14.0
    Uninstalling tensorflow-estimator-1.14.0:
      Successfully uninstalled tensorflow-estimator-1.14.0


## Paso 2: Importar las dependencias

In [2]:
import tensorflow as tf
import numpy as np

from tensorflow.keras.datasets import fashion_mnist

tf.__version__

'1.13.1'

## Paso 3: Pre procesado de datos

### Cargar el dataset de FashionMNIST

In [3]:
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


### Normalizar las imágenes

In [0]:
X_train = X_train / 255.
X_test = X_test / 255.

### Redimensionar las imágenes

In [5]:
X_train.shape

(60000, 28, 28)

In [0]:
X_train = X_train.reshape(-1, 28*28)
X_test = X_test.reshape(-1, 28*28)

In [7]:
X_train.shape

(60000, 784)

## Paso 4: Construir el modelo

### Defining the model

In [0]:
model = tf.keras.models.Sequential()

### Definir el modelo

In [9]:
model.add(tf.keras.layers.Dense(units=128, activation='relu', input_shape=(784,)))
model.add(tf.keras.layers.Dropout(rate=0.2))
model.add(tf.keras.layers.Dense(units=10, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


### Compilar el modelo

In [0]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

### Entrenar el modelo

In [11]:
model.fit(X_train, y_train, epochs=5)

Epoch 1/5
60000/60000 [==============================] - 9s 143us/sample - loss: 0.5281 - sparse_categorical_accuracy: 0.8123
Epoch 2/5
60000/60000 [==============================] - 8s 129us/sample - loss: 0.4006 - sparse_categorical_accuracy: 0.8547
Epoch 3/5
60000/60000 [==============================] - 8s 129us/sample - loss: 0.3681 - sparse_categorical_accuracy: 0.8652
Epoch 4/5
60000/60000 [==============================] - 8s 129us/sample - loss: 0.3463 - sparse_categorical_accuracy: 0.8729
Epoch 5/5
60000/60000 [==============================] - 8s 130us/sample - loss: 0.3323 - sparse_categorical_accuracy: 0.8787


### Evaluar el modelo

In [12]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Precisión del conjunto de test: {}".format(test_accuracy))

10000/10000 [==============================] - 1s 73us/sample - loss: 0.3446 - sparse_categorical_accuracy: 0.8778
Precisión del conjunto de test: 0.8777999877929688


## Paso 5: Convertir el modelo a la versión de TensorFlow Lite

### Guardar el modelo

In [0]:
model_name = 'fashion_mobile_model.h5'
tf.keras.models.save_model(model, model_name)

### Crear el TFLite Converter

In [14]:
converter = tf.lite.TFLiteConverter.from_keras_model_file(model_name)

Instructions for updating:
Use tf.compat.v1.graph_util.convert_variables_to_constants
Instructions for updating:
Use tf.compat.v1.graph_util.extract_sub_graph
INFO:tensorflow:Froze 4 variables.
INFO:tensorflow:Converted 4 variables to const ops.


### Convertir el modelo

In [0]:
tflite_model = converter.convert()

### Guardar la versión de TFLite version del modelo

In [0]:
with open("tf_model.tflite", "wb") as f:
  f.write(tflite_model)